## Altmap Experiments
### Compare altmap to map eq using networkx


In [74]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator)
import numpy as np
from numpy import log2 as ld

# show plots in separate window
%pylab
# load helpers and wrappers
%run helpers.py 

def generate_two_rings(n_ring=10):
    N = 2 * n_ring  # num nodes
    G = nx.MultiGraph()
    G.add_nodes_from(range(1, N + 1))

    for n in range(1, N):
        G.add_edge(n, n + 1, weight=1)

    G.add_edge(1, int(N / 2), weight=1)
    G.add_edge(int(N / 2) + 1, N, weight=1)
    return G


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [73]:
n_ring = 20
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)

# compute altmap cost for different community setups
nodes_ids = list(range(1, N+1))
ring_half1_size = int(n_ring/2)
ring_half2_size = n_ring - ring_half1_size

# initial cost
communities = dict(zip(nodes_ids, nodes_ids))
cost = altmap_cost(G, communities)
print (f'Initial Cost L = {cost}\n')

# ground truth
labels = [1] * n_ring + [2] * n_ring
communities = dict(zip(nodes_ids, labels))
cost = altmap_cost(G, communities)
print (f'Ground Truth Cost L = {cost}\n')

# 2 mixed cliques
labels = [1] * ring_half1_size + [2] * ring_half2_size + [1] * ring_half2_size + [2] * ring_half1_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Mixed Communities Cost L = {cost}\n')

# 4 mixed cliques
labels = [1] * ring_half1_size + [2] * ring_half2_size + [3] * ring_half1_size + [4] * ring_half2_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Four Mixed Communities Cost L = {cost}\n')

Initial Cost L = -0.036833355956923636

Ground Truth Cost L = -0.8412673009000788

Mixed Communities Cost L = -0.4716917484229587

Four Mixed Communities Cost L = -1.28545490767805



In [159]:
# compute essential cost function values for a network of two rings with
# given ring size
def two_rings_cost(ring_size = 3, c1 = 2, c2 = 8, print_output=False):
    n = ring_size
    m = 2*n+1 # number of edges in the network

    J_init = (2*(n-1)*ld(1-1/m)+3*ld(1-3/(2*m)))/m
    J_true = np.log2(m) - 1.0 - (m-1) / m * np.log2(m-1)
    
    alpha = (n%2 != 0) # 1 if odd (3 ind sets), 0 if even (2 ind sets)
    J_ind = -(1 - np.log2(1+2*alpha/m) - 2*alpha/m*np.log2((m-2*alpha)/(m+2*alpha)) - 2*alpha/m)
    
    J_lower_bound = -(3 - 3*np.log2(3) + m*np.log2(m))/m
    
    # c1 modules per ring
    if c1 <= n:
        c=c1
        nm = int(n/c); delta = n-c*nm
        p1=(nm+1)/m; p1not1=1/m; p2=nm/m; p2not2=1/m; p3=(2*nm+1)/(2*m); p3not3=3/(2*m)
        J_c1_comms = 2*(c-delta-1)*altmap_module_cost(p2, p2not2)\
                     + 2*altmap_module_cost(p3, p3not3)
        if delta > 0:
            J_c1_comms += 2*delta*altmap_module_cost(p1, p1not1)
    else:
        J_c1_comms = None
        
    # c2 modules per ring
    if c2 <= n:
        c=c2
        nm = int(n/c); delta = n-c*nm
        p1=(nm+1)/m; p1not1=1/m; p2=nm/m; p2not2=1/m; p3=(2*nm+1)/(2*m); p3not3=3/(2*m)
        J_c2_comms = 2*(c-delta-1)*altmap_module_cost(p2, p2not2)\
                     + 2*altmap_module_cost(p3, p3not3)
        if delta > 0:
            J_c2_comms += 2*delta*altmap_module_cost(p1, p1not1)
    else:
        J_c2_comms = None
        
    if print_output:
        print (f"\n2 Rings network with nc = {n} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}")
        print (f"{c1} communities = {J_c1_comms}")
        print (f"{c2} communities = {J_c2_comms}")
        print (f"Lower bound = {J_lower_bound}\n")
    
    return J_init, J_true, J_ind, J_c1_comms, J_c2_comms

n_max = 30
modules1_per_ring=2
modules2_per_ring=3
n_list = list(range(3, n_max + 1))
J_init_list = np.zeros((len(n_list), 1))
J_true_list = np.zeros((len(n_list), 1))
J_ind_list = np.zeros((len(n_list), 1))
J_c1_comms_list = np.zeros((len(n_list), 1))
J_c2_comms_list = np.zeros((len(n_list), 1))
for i, n in enumerate(n_list):
    J_init_list[i], J_true_list[i], J_ind_list[i], J_c1_comms_list[i],\
    J_c2_comms_list[i] = two_rings_cost(n, c1=modules1_per_ring, c2=modules2_per_ring)

step=1
plt.close('all')
fig, ax = plt.subplots()
plt.suptitle('Network of two rings - objective over ring size')
ax.plot(n_list[::step], -J_true_list[::step], 'x--', label='Ground truth')
ax.plot(n_list[::step], -J_init_list[::step], '^--', label='Each node as a module')
ax.plot(n_list[::step], -J_ind_list[::step], 'o--', label='Maximum independent sets')
ax.plot(n_list[::step], -J_c1_comms_list[::step], 's--', label=f'{modules1_per_ring} modules per ring')
ax.plot(n_list[::step], -J_c2_comms_list[::step], 's--', label=f'{modules2_per_ring} modules per ring')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per ring $n_c$')
ax.set_ylabel('Altmap objective $\mathcal{J}(m)$')
ax.legend()

In [17]:
# generate network
n_ring = 5
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)

# run community detection
# communities_found, num_communities_found = infomap(G, altmap=False)
# communities_found, num_communities_found = infomap(G, altmap=True)
communities_found, num_communities_found = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

cost = altmap_cost(G, communities_found)
print (f'Achieved cost L = {cost}')


OrderedDict([(1, 12), (2, 12), (3, 12), (4, 12), (5, 12), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (12, 7), (13, 7), (14, 7), (15, 7), (16, 7), (17, 7), (18, 7), (19, 7), (20, 3), (21, 3), (22, 3), (23, 3), (24, 3), (25, 3), (26, 3), (27, 3), (28, 3), (29, 3), (30, 3), (31, 3), (32, 3), (33, 3), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 4), (52, 4), (53, 4), (54, 4), (55, 4), (56, 4), (57, 4), (58, 4), (59, 4), (60, 4), (61, 4), (62, 4), (63, 4), (64, 4), (65, 4), (66, 2), (67, 2), (68, 2), (69, 2), (70, 2), (71, 2), (72, 2), (73, 2), (74, 2), (75, 2), (76, 2), (77, 2), (78, 2), (79, 2), (80, 2), (81, 2), (82, 5), (83, 5), (84, 5), (85, 5), (86, 5), (87, 5), (88, 5), (89, 5), (90, 5), (91, 5), (92, 5), (93, 5), (94, 5), (95, 5), (96, 5), (97, 12), (98, 12), (99, 12), (100, 12), (101, 12), (102, 8), (103, 8), (104, 8), (105, 8), (106, 8), (107, 8), (108, 8), (109,

In [18]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)

In [158]:
# generate network
n_ring = 10
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)
nodes_ids = list(range(1, N+1))
ring_half1_size = int(n_ring/2)
ring_half2_size = n_ring - ring_half1_size

n_ring_odd = n_ring+1
G_odd = generate_two_rings(n_ring_odd)
nodes_ids_odd = list(range(1, 2*n_ring_odd+1))

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * n_ring + [2] * n_ring
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = [1,2] * n_ring
communities_ind_even = dict(zip(nodes_ids, labels))

# labels = [1,2] * int(n_ring/2) + [3] + [1,2] * int(n_ring/2) + [3]
labels = [1,2] * (int(n_ring_odd/2) - 1) + [1,3,2] + [1,2] * int(n_ring_odd/2) + [3]
communities_ind_odd = dict(zip(nodes_ids_odd, labels))

# c modules per ring
c=3
nm = int(n_ring/c); delta = n_ring-c*nm
labels = [module for module in range(1,delta+1) for i in range(0,nm+1)]
labels.extend([module for module in range(delta+1,c+1) for i in range(0,nm)])
labels.extend([module for module in range(c+1,2*c-delta+1) for i in range(0,nm)])
labels.extend([module for module in range(2*c-delta+1,2*c+1) for i in range(0,nm+1)])
communities_c_modules = dict(zip(nodes_ids, labels))

plt.close('all')
fig, axs = plt.subplots(1,5)
fig.suptitle(f'Sample two rings networks')

drawNetwork(G, communities_true, ax=axs[0])
axs[0].set_xlabel('Ground truth')
drawNetwork(G, communities_init, ax=axs[1])
axs[1].set_xlabel('Initial partition')
drawNetwork(G, communities_ind_even, ax=axs[2])
axs[2].set_xlabel('Max independent sets\n$n_c$ even')
drawNetwork(G_odd, communities_ind_odd, ax=axs[3])
axs[3].set_xlabel('Max independent sets\n$n_c$ odd')
drawNetwork(G, communities_c_modules, ax=axs[4])
axs[4].set_xlabel(f'{c} modules per ring')

/home/chri/anaconda3/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


Text(0.5, 0, '3 modules per ring')